In [1]:
import open3d as o3d
import jittor as jt
import numpy as np
import mcubes
import trimesh
from plyfile import PlyData, PlyElement
from jnerf.runner import Runner 
from jnerf.utils.config import init_cfg
init_cfg('./projects/ngp/configs/ngp_base.py')
runner = Runner()
runner.load_ckpt(runner.ckpt_path)
N = 128
xmin, xmax = 0, 1
ymin, ymax = 0, 1
zmin, zmax = 0, 1
x = jt.linspace(xmin, xmax, N)
y = jt.linspace(ymin, ymax, N)
z = jt.linspace(zmin, zmax, N)
xyz_ = jt.stack(jt.meshgrid(x, y, z), -1).reshape(-1, 3)
dir_ = jt.zeros_like(xyz_)
with jt.no_grad():
    B = xyz_.shape[0]
    out_chunks = []
    for i in range(0, B, runner.n_rays_per_batch):
        out_chunks += [runner.model(xyz_[i:i + runner.n_rays_per_batch], dir_[i:i + runner.n_rays_per_batch])]
    rgbsigma = np.concatenate(out_chunks, 0)
sigma = rgbsigma[:, -1]
sigma = np.maximum(sigma, 0)
sigma = sigma.reshape(N, N, N)
sigma = sigma.astype(int)
vertices, triangles = mcubes.marching_cubes(sigma, 0.5)
mesh_o = trimesh.Trimesh(vertices / N, triangles)
mesh_o.show()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


[i 0908 16:26:42.594700 96 log.cc:351] Load log_sync: 1
[i 0908 16:26:42.682375 96 compiler.py:955] Jittor(1.3.4.17) src: /home/featurize/work/.local/lib/python3.7/site-packages/jittor
[i 0908 16:26:42.695858 96 compiler.py:956] g++ at /usr/bin/g++(9.3.0)
[i 0908 16:26:42.697890 96 compiler.py:957] cache_path: /home/featurize/.cache/jittor/jt1.3.4/g++9.3.0/py3.7.10/Linux-5.4.0-91xc3/IntelRXeonRGolx46/default
[i 0908 16:26:42.711983 96 __init__.py:411] Found nvcc(11.2.152) at /usr/local/cuda/bin/nvcc.
[i 0908 16:26:42.727839 96 __init__.py:411] Found addr2line(2.34) at /usr/bin/addr2line.
[i 0908 16:26:42.894717 96 compiler.py:1010] cuda key:cu11.2.152_sm_86
[i 0908 16:26:43.251681 96 __init__.py:227] Total mem: 55.60GB, using 16 procs for compiling.
[i 0908 16:26:43.672010 96 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0908 16:26:43.784735 96 init.cc:62] Found cuda archs: [86,]
[i 0908 16:26:44.185255 96 compile_extern.py:516] mpicc not found, distribution disabled.
[i 0908 16:26

Loading config from:  ./projects/ngp/configs/ngp_base.py
load train data


100%|██████████| 400/400 [00:17<00:00, 22.71it/s]


load val data


100%|██████████| 20/20 [00:03<00:00,  5.99it/s]


Loading ckpt from: ./logs/lego/params.pkl


[e 0908 16:27:13.416270 96 __init__.py:1351] load parameter exposure failed: expect the shape of exposure to be [1200,], but got [600,]
[w 0908 16:27:13.495053 96 __init__.py:1354] load total 12 params, 1 failed


In [3]:
vertices_ = (vertices/N).astype(np.float32)
x_ = (ymax - ymin) * vertices_[:, 1] + ymin
y_ = (xmax - xmin) * vertices_[:, 0] + xmin
vertices_[:, 0] = x_
vertices_[:, 1] = y_
vertices_[:, 2] = (zmax - zmin) * vertices_[:, 2] + zmin
vertices_.dtype = [('x', 'f4'), ('y','f4'), ('z', 'f4')]
face = np.empty(len(triangles), dtype=[('vertex_indices', 'i4', (3,))])
face['vertex_indices'] = triangles
print("mesh 已经生成 mesh.ply")
PlyData([PlyElement.describe(vertices_[:, 0], 'vertex'), 
        PlyElement.describe(face, 'face')]).write(f'{"mesh"}.ply')
mesh = o3d.io.read_triangle_mesh(f"{'mesh'}.ply")
idxs, count, _ = mesh.cluster_connected_triangles()
max_cluster_idx = np.argmax(count)
triangles_to_remove = [i for i in range(len(face)) if idxs[i] != max_cluster_idx]
mesh.remove_triangles_by_index(triangles_to_remove)
mesh.remove_unreferenced_vertices()
face = np.empty(len(mesh.triangles), dtype=[('vertex_indices', 'i4', (3,))])
face['vertex_indices'] = mesh.triangles
vertices_ = np.asarray(mesh.vertices).astype(np.float32)
vertices_.dtype = [('x', 'f4'), ('y', 'f4'), ('z', 'f4')]
PlyData([PlyElement.describe(vertices_[:, 0], 'vertex'), 
        PlyElement.describe(face, 'face')]).write(f'{"mesh-noise"}.ply')
print("mesh 去噪 已经生成 mesh-noise.ply")

mesh 已经生成 mesh.ply
mesh 去噪 已经生成 mesh-noise.ply
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.
